<a href="https://colab.research.google.com/github/diegoavarela/colab-tests/blob/main/Users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Return users


In [4]:
def new_returning_users(users):
  
  new = 0
  returning = 0  
  
  if users.ndim == 2:
    returning = users[1]
    new = users[0]
  else:
    returning = users

  return  [new , returning]

# Read File


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

dic_users = { 1: 'New User', 2: 'Returning User'}

users = pd.read_csv('/content/support_files/New vs Returning - Connected Sheet 1.csv')

users_sliced = users.copy()

users_sliced[ 'Amount Users'] = 0
users_sliced = users_sliced[['user_type','client', 'Amount Users']]
users_sliced = users_sliced.groupby([ 'user_type', 'client'])['Amount Users'].count().reset_index()
users_sliced = users_sliced.sort_values(by='Amount Users', ascending=False).reset_index()

#users_sliced = users_sliced.head(20)

plt.figure(figsize=(8,10))

# bar chart 1 -> top bars (group of 'smoker=No')
bar1 = sns.barplot(x="Amount Users",  y="client", data=users_sliced, color='darkblue', orient='h')



# bottom bar ->  take only smoker=Yes values from the data
final = users_sliced[users_sliced.user_type == dic_users[1]]


# bar chart 2 -> bottom bars (group of 'smoker=Yes')
bar2 = sns.barplot(x="Amount Users", y="client", data=final, estimator=sum, ci=100,  color='lightblue', orient='h')


# add legend
top_bar = mpatches.Patch(color='darkblue', label='Returning Users')
bottom_bar = mpatches.Patch(color='lightblue', label='New Users')
plt.legend(handles=[top_bar, bottom_bar])